# Create an acoustic indices representation of audio
ToDo: You need to adapt to your own environment



## Loading the audio

In [ ]:
# ToDo
# Ex. Mount Google Drive
#   from google.colab import drive
#   drive.mount('/content/drive')

Mounted at /content/drive


## Setting up the coding environment

### Installing external packages

In [ ]:
!pip install scikit-maad

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.4/142.4 KB 5.1 MB/s eta 0:00:00


### Importing libraries

In [ ]:
import os
import sys
import time
import glob
import scipy.io.wavfile
import scipy.signal
import numpy as np
import pandas as pd
import librosa
import datetime
from IPython.display import Audio
from sklearn.preprocessing import OneHotEncoder
from maad import sound, features
from maad.util import (date_parser, plot_correlation_map,
                       plot_features_map, plot_features, false_Color_Spectro)

### Defining global variables

In [ ]:
# List all spectral and temporal acoustic indices that will be computed
SPECTRAL_FEATURES=['MEANf','VARf','SKEWf','KURTf','NBPEAKS','LEQf',
'ENRf','BGNf','SNRf','Hf', 'EAS','ECU','ECV','EPS','EPS_KURT','EPS_SKEW','ACI',
'NDSI','rBA','AnthroEnergy','BioEnergy','BI','ROU','ADI','AEI','LFC','MFC','HFC',
'ACTspFract','ACTspCount','ACTspMean', 'EVNspFract','EVNspMean','EVNspCount',
'TFSD','H_Havrda','H_Renyi','H_pairedShannon', 'H_gamma', 'H_GiniSimpson','RAOQ',
'AGI','ROItotal','ROIcover']

TEMPORAL_FEATURES=['ZCR','MEANt', 'VARt', 'SKEWt', 'KURTt',
               'LEQt','BGNt', 'SNRt','MED', 'Ht','ACTtFraction', 'ACTtCount',
               'ACTtMean','EVNtFraction', 'EVNtMean', 'EVNtCount']

In [ ]:
# ToDo
# Ex. Audio dataset location
#   parent_dir = '/content/drive/MyDrive/Volet1'
#   label_dirs = ['RE', 'BE', 'BL', 'RL']

## Initializing the environment 

### Getting the number of files to process per label


In [ ]:
# Get the number of audio files in each label directory
num_audio_files = {}
for label in label_dirs:
    label_dir = os.path.join(parent_dir, label)
    audio_files = glob.glob(label_dir + '/*/*.wav')
    num_audio_files[label] = len(audio_files)
num_audio_files

{'RE': 3, 'BE': 3, 'BL': 3, 'RL': 3}

### Defining auxiliary functions

In [ ]:
def audio_preprocessing(audio, sr, new_sr, chunk_size, total_size):
    """Preprocess the audio file
    1. Resample the audio file
    2. Pad the audio file to 30 minutes
    3. Split the audio into chunks of 60 seconds
    
    Args:
        audio (np.array): audio signal
        sr (int): sampling rate
        new_sr (int): new sampling rate
        chunk_size (int): chunk size in seconds
        total_size (int): total size in minutes
    
    Returns:
        audio_chunks (np.array): audio chunks
        """
    # Resample the audio file
    audio = librosa.resample(audio, sr, new_sr)
    # Pad the audio file to 30 minutes
    audio = np.pad(audio, (0, total_size*chunk_size*new_sr - audio.shape[0]), 'constant')
    # Split the audio into chunks of 60 seconds
    audio_chunks = librosa.util.frame(audio, frame_length=chunk_size*new_sr, hop_length=chunk_size*new_sr)
    return audio_chunks

In [ ]:
def date_time_parser_from_filename(filename):
    """Parse the date and time from the filename
    
    Args:
        filename (str): filename (filename format: XXX_YYYYMMDD_HHMMSS.wav)
    
    Returns:
        date_time_obj (obj): date + time
        """
    # Get the date and time from the filename
    date_time = filename.split('_')[1] + filename.split('_')[2].split('.')[0]
    # Convert the date and time to datetime object
    date_time_obj = datetime.datetime.strptime(date_time, '%Y%m%d%H%M%S')
    return date_time_obj

## Computing acoustic indices 



In [ ]:
# Create a dataframe to store the features
df_indices = pd.DataFrame(columns=['filename']+['date']+['site']+TEMPORAL_FEATURES+SPECTRAL_FEATURES+['label'])
# Parameters
new_sr = 16000 # ToDo: new sampling rate
chunk_size = 60 # ToDo: 60 seconds
total_size = 30 # ToDo: 30 minutes

# Parameters of the audio recorder. This is not a mandatory but it allows
# to compute the sound pressure level of the audio file (dB SPL) as a
# sonometer would do.
S = -35         # ToDo: Sensbility microphone-35dBV (SM4) / -18dBV (Audiomoth)
G = 26+16       # ToDo: Amplification gain (26dB (SM4 preamplifier))

# Loop through the label directories and load the audio files
for label in label_dirs:
    # Print the label directory
    print('Loading label directory: {}'.format(label))
    # Get the path to the label directory
    label_dir = os.path.join(parent_dir, label)
    # Get the list of audio files
    audio_files = glob.glob(label_dir + '/*/*.wav')
    # Loop through the audio files
    for audio_file in audio_files:
        # Initialize start time
        start_time = time.time()
        # Get the filename
        filename = os.path.basename(audio_file)
        # Get the date and time from the filename
        date = date_time_parser_from_filename(filename)
        # Get the site lowercase (parent directory name)
        site = os.path.basename(os.path.dirname(audio_file)).lower()
        # Load the audio file
        audio, sr = librosa.load(audio_file)
        # Preprocess the audio file
        audio_chunks = audio_preprocessing(audio, sr, new_sr, chunk_size, total_size)
        # Loop through the audio chunks
        for audio_chunk in audio_chunks.T:
            # Compute the acoustic indices in the time domain
            df_temp_ind = features.all_temporal_alpha_indices(audio_chunk, new_sr, gain = G, sensibility = S, 
                                          dB_threshold = 3, rejectDuration = 0.01,
                                          verbose = False, display = False)
            # Compute the Power Spectrogram Density (PSD) : Sxx_power
            Sxx_power,tn,fn,ext = sound.spectrogram (audio_chunk, new_sr)
            # Compute the acoustic indices in the frequency domain
            df_freq_ind, _ = features.all_spectral_alpha_indices(Sxx_power, tn, fn)
            # Concatenate the acoustic indices in the time and frequency domain
            df_ind = pd.concat([df_temp_ind, df_freq_ind], axis=1)
            # Add the filename, date, site and label to the dataframe
            df_ind['filename'] = filename
            df_ind['date'] = date
            df_ind['site'] = site
            df_ind['label'] = label
            # Append the dataframe to the dataframe containing all the acoustic indices
            df_indices = df_indices.append(df_ind, ignore_index=True)
        # Calculate and print the total time taken for the loop
        end_time = time.time()
        total_time = end_time - start_time
        # Print the progress of the loop compare to the total number of audio files
        print('Loading audio file: {}/{} Total time taken: {} seconds'.format(audio_files.index(audio_file)+1, num_audio_files[label], total_time))

Loading label directory: RE


/usr/local/lib/python3.8/dist-packages/maad/util/math_func.py:445: RuntimeWarning: divide by zero encountered in log
  H = -np.sum(pmf*np.log(pmf),axis)/np.log(n)
/usr/local/lib/python3.8/dist-packages/maad/util/math_func.py:445: RuntimeWarning: invalid value encountered in multiply
  H = -np.sum(pmf*np.log(pmf),axis)/np.log(n)
/usr/local/lib/python3.8/dist-packages/maad/util/miscellaneous.py:358: RuntimeWarning: divide by zero encountered in log10
  y = 20*log10(x)   # take log
/usr/local/lib/python3.8/dist-packages/maad/spl/conversion_SPL.py:478: RuntimeWarning: divide by zero encountered in log10
  Leq = 20*log10(volt_RMS) - sensitivity + dBref - gain
/usr/local/lib/python3.8/dist-packages/maad/util/miscellaneous.py:413: RuntimeWarning: divide by zero encountered in log10
  y = 10*log10(x)   # take log
/usr/local/lib/python3.8/dist-packages/maad/features/alpha_indices.py:768: RuntimeWarning: divide by zero encountered in log10
  envdB = 10*np.log10(env**2)


Loading audio file: 1/3 Total time taken: 64.21057844161987 seconds
Loading audio file: 2/3 Total time taken: 59.32668423652649 seconds
Loading audio file: 3/3 Total time taken: 58.86629939079285 seconds
Loading label directory: BE
Loading audio file: 1/3 Total time taken: 55.64867067337036 seconds
Loading audio file: 2/3 Total time taken: 56.18789529800415 seconds
Loading audio file: 3/3 Total time taken: 55.564722537994385 seconds
Loading label directory: BL
Loading audio file: 1/3 Total time taken: 53.69717454910278 seconds
Loading audio file: 2/3 Total time taken: 59.635740756988525 seconds
Loading audio file: 3/3 Total time taken: 56.594600200653076 seconds
Loading label directory: RL
Loading audio file: 1/3 Total time taken: 54.175310373306274 seconds
Loading audio file: 2/3 Total time taken: 55.97605347633362 seconds
Loading audio file: 3/3 Total time taken: 56.40797758102417 seconds


## Saving the dataset

In [ ]:
# ToDo
# Ex. Save the dataframe to a csv file
#   path = '/content/drive/MyDrive/dataset'
#   dataset_filename = 'indices.csv'
#   df_indices.to_csv(os.path.join(path, dataset_filename), index=False)